Import needed modules

In [2]:
import pandas as pd
import os
import subprocess

Import data

In [23]:
# read the data
data_LmSIDERs = pd.read_csv("./data/2.SIDERs_filter/positives_testing_elements.csv", sep=",", header=None)
data_LmSIDERs.columns = ["sseqid", "length", "sstart", "send", "sstrand", "sseq"]
data_chr32 = pd.read_csv("./data/4.SIDERs_filter_32chr/positives_testing_elements.csv", sep=",", header=None)
data_chr32.columns = ["sseqid", "length", "sstart", "send", "sstrand", "sseq"]
print(f"Output from LmSIDERs row length: {data_LmSIDERs.shape[0]}\n",
      f"Output from chr 32 row length: {data_chr32.shape[0]}\n",
      f"Difference: {abs(data_LmSIDERs.shape[0] - data_chr32.shape[0])}",
      sep="")

Output from LmSIDERs row length: 2156
Output from chr 32 row length: 2184
Difference: 28


In [24]:
# Declare bedops folders paths:
bedops_path_1 = "./bedops_compare"
bedops_path_2 = "./bedops_compare_inverse"


Check how many elements for each chromosome in the "True Positive" data

In [16]:
data_LmSIDERs["sseqid"].value_counts(sort=False)

sseqid
LinJ.01     13
LinJ.02     16
LinJ.03      6
LinJ.04     13
LinJ.05     21
LinJ.06     26
LinJ.07     31
LinJ.08     39
LinJ.09     32
LinJ.10     43
LinJ.11     42
LinJ.12     46
LinJ.13     35
LinJ.14     42
LinJ.15     27
LinJ.16     38
LinJ.17     48
LinJ.18     56
LinJ.19     58
LinJ.20     48
LinJ.21     43
LinJ.22     60
LinJ.23     73
LinJ.24     56
LinJ.25     38
LinJ.26     56
LinJ.27     81
LinJ.28     59
LinJ.29    122
LinJ.30    109
LinJ.31    110
LinJ.32     95
LinJ.33    116
LinJ.34    156
LinJ.35    143
LinJ.36    159
Name: count, dtype: int64

Check how many elements for each chromosome in the software data

In [17]:
data_chr32["sseqid"].value_counts(sort=False)

sseqid
LinJ.01     13
LinJ.02     16
LinJ.03      8
LinJ.04     12
LinJ.05     22
LinJ.06     28
LinJ.07     32
LinJ.08     38
LinJ.09     33
LinJ.10     44
LinJ.11     42
LinJ.12     47
LinJ.13     33
LinJ.14     43
LinJ.15     27
LinJ.16     38
LinJ.17     50
LinJ.18     59
LinJ.19     58
LinJ.20     49
LinJ.21     42
LinJ.22     62
LinJ.23     76
LinJ.24     54
LinJ.25     41
LinJ.26     53
LinJ.27     79
LinJ.28     59
LinJ.29    127
LinJ.30    106
LinJ.31    115
LinJ.32     95
LinJ.33    116
LinJ.34    159
LinJ.35    145
LinJ.36    163
Name: count, dtype: int64

Difference in numbers in the software data compared to the "True Positives"

In [25]:
data_LmSIDERs["sseqid"].value_counts(sort=False) - data_chr32["sseqid"].value_counts(sort=False)

sseqid
LinJ.01    0
LinJ.02    0
LinJ.03   -2
LinJ.04    1
LinJ.05   -1
LinJ.06   -2
LinJ.07   -1
LinJ.08    1
LinJ.09   -1
LinJ.10   -1
LinJ.11    0
LinJ.12   -1
LinJ.13    2
LinJ.14   -1
LinJ.15    0
LinJ.16    0
LinJ.17   -2
LinJ.18   -3
LinJ.19    0
LinJ.20   -1
LinJ.21    1
LinJ.22   -2
LinJ.23   -3
LinJ.24    2
LinJ.25   -3
LinJ.26    3
LinJ.27    2
LinJ.28    0
LinJ.29   -5
LinJ.30    3
LinJ.31   -5
LinJ.32    0
LinJ.33    0
LinJ.34   -3
LinJ.35   -2
LinJ.36   -4
Name: count, dtype: int64

Let's group data by chromosomes

In [30]:
data_LmSIDERs_grouped = data_LmSIDERs.groupby("sseqid")
data_chr32_grouped = data_chr32.groupby("sseqid")

Let's make the main code

In [31]:
# Check actual path
os.getcwd()

'/home/rfpacheco/Documents/Work_CBMSO/Leish_Data_Analysis/14.From_all_to_real_SIDERs'

In [32]:
def columns_to_numeric(data_input, columns_to_convert = []):
    for column in columns_to_convert:
        data_input[column] = pd.to_numeric(data_input[column], errors='coerce')
    return data_input

Elements from LmSIDERs data frame found in the data frame from the chromosome 32

In [37]:
# Folder path, change it
# bedops_path = "./bedops_data2"

comparison_dict_1 = {}
for (name1, group1), (name2, group2) in zip(data_LmSIDERs_grouped, data_chr32_grouped):
    # group1 should be the TP data
    # ------------------------------------------------------------------------------
    path_chr = os.path.join(bedops_path_2, name1)
    os.makedirs(path_chr, exist_ok=True)
    # ------------------------------------------------------------------------------
    group1_total = group1[["sseqid", "sstart", "send"]].copy()
    group2_total = group2[["sseqid", "sstart", "send"]].copy()
    # ------------------------------------------------------------------------------
    group1_total.sort_values(by=["sstart", "send"], inplace=True)
    group2_total.sort_values(by=["sstart", "send"], inplace=True)
    # ------------------------------------------------------------------------------
    path_group1_total = os.path.join(path_chr, "group1_total.bed")
    path_group2_total = os.path.join(path_chr, "group2_total.bed")
    # ------------------------------------------------------------------------------
    group1_total.to_csv(path_group1_total, sep="\t", header=False, index=False)  # tabular sep because of bed format
    group2_total.to_csv(path_group2_total, sep="\t", header=False, index=False)  # tabular sep because of bed format

    # ------------------------------------------------------------------------------
    condition = 1
    cmd = f"bedops --element-of {condition} {path_group1_total} {path_group2_total}"
    result = subprocess.check_output(cmd, shell=True, universal_newlines=True)
    result_df = pd.DataFrame([x.split("\t") for x in result.split("\n") if x], 
                                columns=["sseqid", "sstart", "send"])
    result_df = columns_to_numeric(result_df, ["sstart", "send"])  # defined function
    comparison_dict_1[name1] = [f"{result_df.shape[0]}/{group1_total.shape[0]}", f"{result_df.shape[0]/group1_total.shape[0]*100:.2f}"]
comparison_dict_1

{'LinJ.01': ['13/13', '100.00'],
 'LinJ.02': ['16/16', '100.00'],
 'LinJ.03': ['6/6', '100.00'],
 'LinJ.04': ['12/13', '92.31'],
 'LinJ.05': ['21/21', '100.00'],
 'LinJ.06': ['26/26', '100.00'],
 'LinJ.07': ['31/31', '100.00'],
 'LinJ.08': ['39/39', '100.00'],
 'LinJ.09': ['32/32', '100.00'],
 'LinJ.10': ['43/43', '100.00'],
 'LinJ.11': ['42/42', '100.00'],
 'LinJ.12': ['46/46', '100.00'],
 'LinJ.13': ['35/35', '100.00'],
 'LinJ.14': ['42/42', '100.00'],
 'LinJ.15': ['27/27', '100.00'],
 'LinJ.16': ['38/38', '100.00'],
 'LinJ.17': ['47/48', '97.92'],
 'LinJ.18': ['56/56', '100.00'],
 'LinJ.19': ['58/58', '100.00'],
 'LinJ.20': ['48/48', '100.00'],
 'LinJ.21': ['42/43', '97.67'],
 'LinJ.22': ['60/60', '100.00'],
 'LinJ.23': ['72/73', '98.63'],
 'LinJ.24': ['53/56', '94.64'],
 'LinJ.25': ['38/38', '100.00'],
 'LinJ.26': ['54/56', '96.43'],
 'LinJ.27': ['81/81', '100.00'],
 'LinJ.28': ['59/59', '100.00'],
 'LinJ.29': ['119/122', '97.54'],
 'LinJ.30': ['109/109', '100.00'],
 'LinJ.31': ['1

Elements from the output of chr 32 found in LmSIDERs

In [38]:
# Folder path, change it
# bedops_path = "./bedops_data2"

comparison_dict_2 = {}
for (name1, group1), (name2, group2) in zip(data_chr32_grouped, data_LmSIDERs_grouped):
    # group1 should be the TP data
    # ------------------------------------------------------------------------------
    path_chr = os.path.join(bedops_path_2, name1)
    os.makedirs(path_chr, exist_ok=True)
    # ------------------------------------------------------------------------------
    group1_total = group1[["sseqid", "sstart", "send"]].copy()
    group2_total = group2[["sseqid", "sstart", "send"]].copy()
    # ------------------------------------------------------------------------------
    group1_total.sort_values(by=["sstart", "send"], inplace=True)
    group2_total.sort_values(by=["sstart", "send"], inplace=True)
    # ------------------------------------------------------------------------------
    path_group1_total = os.path.join(path_chr, "group1_total.bed")
    path_group2_total = os.path.join(path_chr, "group2_total.bed")
    # ------------------------------------------------------------------------------
    group1_total.to_csv(path_group1_total, sep="\t", header=False, index=False)  # tabular sep because of bed format
    group2_total.to_csv(path_group2_total, sep="\t", header=False, index=False)  # tabular sep because of bed format

    # ------------------------------------------------------------------------------
    condition = 1
    cmd = f"bedops --element-of {condition} {path_group1_total} {path_group2_total}"
    result = subprocess.check_output(cmd, shell=True, universal_newlines=True)
    result_df = pd.DataFrame([x.split("\t") for x in result.split("\n") if x], 
                                columns=["sseqid", "sstart", "send"])
    result_df = columns_to_numeric(result_df, ["sstart", "send"])  # defined function
    comparison_dict_2[name1] = [f"{result_df.shape[0]}/{group1_total.shape[0]}", f"{result_df.shape[0]/group1_total.shape[0]*100:.2f}"]
comparison_dict_2

{'LinJ.01': ['13/13', '100.00'],
 'LinJ.02': ['16/16', '100.00'],
 'LinJ.03': ['6/8', '75.00'],
 'LinJ.04': ['12/12', '100.00'],
 'LinJ.05': ['21/22', '95.45'],
 'LinJ.06': ['28/28', '100.00'],
 'LinJ.07': ['32/32', '100.00'],
 'LinJ.08': ['38/38', '100.00'],
 'LinJ.09': ['33/33', '100.00'],
 'LinJ.10': ['44/44', '100.00'],
 'LinJ.11': ['42/42', '100.00'],
 'LinJ.12': ['46/47', '97.87'],
 'LinJ.13': ['32/33', '96.97'],
 'LinJ.14': ['43/43', '100.00'],
 'LinJ.15': ['27/27', '100.00'],
 'LinJ.16': ['38/38', '100.00'],
 'LinJ.17': ['50/50', '100.00'],
 'LinJ.18': ['58/59', '98.31'],
 'LinJ.19': ['58/58', '100.00'],
 'LinJ.20': ['49/49', '100.00'],
 'LinJ.21': ['42/42', '100.00'],
 'LinJ.22': ['62/62', '100.00'],
 'LinJ.23': ['75/76', '98.68'],
 'LinJ.24': ['54/54', '100.00'],
 'LinJ.25': ['40/41', '97.56'],
 'LinJ.26': ['53/53', '100.00'],
 'LinJ.27': ['79/79', '100.00'],
 'LinJ.28': ['59/59', '100.00'],
 'LinJ.29': ['124/127', '97.64'],
 'LinJ.30': ['106/106', '100.00'],
 'LinJ.31': ['11

In [45]:
for (key1, values1), (key2, values2) in zip(comparison_dict_1.items(), comparison_dict_2.items()):
    if values1[1] != "100.00" or values2[1] != "100.00" or values1[0].split("/")[1] != values2[0].split("/")[1]:
        print(f"Name: {key1}\n",
              f"\t- LmSIDERs in Chr32: {values1[0]} ({values1[1]}%)\n",
              f"\t- Chr32 in LmSIDERs: {values2[0]} ({values2[1]}%)\n",
              sep="")

Name: LinJ.03
	- LmSIDERs in Chr32: 6/6 (100.00%)
	- Chr32 in LmSIDERs: 6/8 (75.00%)

Name: LinJ.04
	- LmSIDERs in Chr32: 12/13 (92.31%)
	- Chr32 in LmSIDERs: 12/12 (100.00%)

Name: LinJ.05
	- LmSIDERs in Chr32: 21/21 (100.00%)
	- Chr32 in LmSIDERs: 21/22 (95.45%)

Name: LinJ.06
	- LmSIDERs in Chr32: 26/26 (100.00%)
	- Chr32 in LmSIDERs: 28/28 (100.00%)

Name: LinJ.07
	- LmSIDERs in Chr32: 31/31 (100.00%)
	- Chr32 in LmSIDERs: 32/32 (100.00%)

Name: LinJ.08
	- LmSIDERs in Chr32: 39/39 (100.00%)
	- Chr32 in LmSIDERs: 38/38 (100.00%)

Name: LinJ.09
	- LmSIDERs in Chr32: 32/32 (100.00%)
	- Chr32 in LmSIDERs: 33/33 (100.00%)

Name: LinJ.10
	- LmSIDERs in Chr32: 43/43 (100.00%)
	- Chr32 in LmSIDERs: 44/44 (100.00%)

Name: LinJ.12
	- LmSIDERs in Chr32: 46/46 (100.00%)
	- Chr32 in LmSIDERs: 46/47 (97.87%)

Name: LinJ.13
	- LmSIDERs in Chr32: 35/35 (100.00%)
	- Chr32 in LmSIDERs: 32/33 (96.97%)

Name: LinJ.14
	- LmSIDERs in Chr32: 42/42 (100.00%)
	- Chr32 in LmSIDERs: 43/43 (100.00%)

Name: Li

In [49]:
Counter_LmSIDERs = 0
Counter_chr32 = 0
for (key1, values1), (key2, values2) in zip(comparison_dict_1.items(), comparison_dict_2.items()):
    if values1[1] != "100.00" or values2[1] != "100.00":
        if values1[1] != "100.00":
            number = int(values1[0].split("/")[1]) - int(values1[0].split("/")[0])
            Counter_LmSIDERs += number
        if values2[1] != "100.00":
            number = int(values2[0].split("/")[1]) - int(values2[0].split("/")[0])
            Counter_chr32 += number
        print(f"Name: {key1}\n",
              f"\t- LmSIDERs in Chr32: {values1[0]} ({values1[1]}%)\n",
              f"\t- Chr32 in LmSIDERs: {values2[0]} ({values2[1]}%)\n",
              sep="")

Name: LinJ.03
	- LmSIDERs in Chr32: 6/6 (100.00%)
	- Chr32 in LmSIDERs: 6/8 (75.00%)

Name: LinJ.04
	- LmSIDERs in Chr32: 12/13 (92.31%)
	- Chr32 in LmSIDERs: 12/12 (100.00%)

Name: LinJ.05
	- LmSIDERs in Chr32: 21/21 (100.00%)
	- Chr32 in LmSIDERs: 21/22 (95.45%)

Name: LinJ.12
	- LmSIDERs in Chr32: 46/46 (100.00%)
	- Chr32 in LmSIDERs: 46/47 (97.87%)

Name: LinJ.13
	- LmSIDERs in Chr32: 35/35 (100.00%)
	- Chr32 in LmSIDERs: 32/33 (96.97%)

Name: LinJ.17
	- LmSIDERs in Chr32: 47/48 (97.92%)
	- Chr32 in LmSIDERs: 50/50 (100.00%)

Name: LinJ.18
	- LmSIDERs in Chr32: 56/56 (100.00%)
	- Chr32 in LmSIDERs: 58/59 (98.31%)

Name: LinJ.21
	- LmSIDERs in Chr32: 42/43 (97.67%)
	- Chr32 in LmSIDERs: 42/42 (100.00%)

Name: LinJ.23
	- LmSIDERs in Chr32: 72/73 (98.63%)
	- Chr32 in LmSIDERs: 75/76 (98.68%)

Name: LinJ.24
	- LmSIDERs in Chr32: 53/56 (94.64%)
	- Chr32 in LmSIDERs: 54/54 (100.00%)

Name: LinJ.25
	- LmSIDERs in Chr32: 38/38 (100.00%)
	- Chr32 in LmSIDERs: 40/41 (97.56%)

Name: LinJ.26
	

In [50]:
print(f"Elements in LmSIDERs but not in Chr32: {Counter_LmSIDERs}\n",
      f"Elements in Chr32 but not in LmSIDERs: {Counter_chr32}",
      sep="")

Elements in LmSIDERs but not in Chr32: 20
Elements in Chr32 but not in LmSIDERs: 15
